In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.firefox.options import Options

from bs4 import BeautifulSoup
import re
import pandas as pd
import os

import time 

## Remember to download latest selenium, Firefox, and geckodriver from Github. Run geckodriver in the background

In [2]:
import itertools

def xpath_soup(element):
    """
    Generate xpath of soup element
    :param element: bs4 text or node
    :return: xpath as string
    """
    components = []
    child = element if element.name else element.parent
    for parent in child.parents:
        """
        @type parent: bs4.element.Tag
        """
        previous = itertools.islice(parent.children, 0, parent.contents.index(child))
        xpath_tag = child.name
        xpath_index = sum(1 for i in previous if i.name == xpath_tag) + 1
        components.append(xpath_tag if xpath_index == 1 else '%s[%d]' % (xpath_tag, xpath_index))
        child = parent
    components.reverse()
    return '/%s' % '/'.join(components)

In [3]:
#launch url

url ="http://www.weather.gov.sg/climate-historical-daily"
places = ['Sembawang','Paya Lebar','Changi','Tengah']
sel_years = list(range(2011,2019))

# set firefox preferences
fp = webdriver.FirefoxProfile()
fp.set_preference('browser.download.folderList',2)
fp.set_preference('browser.download.manager.showWhenStarting',False)
mime_types = [
    'text/plain', 
    'application/vnd.ms-excel', 
    'text/csv', 
    'application/csv', 
    'text/comma-separated-values', 
    'application/download', 
    'application/octet-stream', 
    'binary/octet-stream', 
    'application/binary', 
    'application/x-unknown'
]
fp.set_preference("browser.helperApps.neverAsk.saveToDisk", ",".join(mime_types))

# create a new Firefox session
driver = webdriver.Firefox(firefox_profile=fp)
driver.implicitly_wait(30)
driver.get(url)

In [4]:
for p in places:
    city_btn = driver.find_element_by_id('cityname')
    
    soup_level1=BeautifulSoup(driver.page_source,'lxml')
    citylist = soup_level1.find_all('button',{'id':'cityname'})[0].parent.find_all('li')
    index = [idx for idx, s in enumerate(citylist) if p in s.text][0]
    citylink = driver.find_element_by_xpath(xpath_soup(citylist[index]))
    city_btn.click()
    citylink.click()
        
    for y in sel_years:
        print(y)
        year_btn = driver.find_element_by_id('year')
        
        yearlist = soup_level1.find_all('button',{'id':'year'})[0].parent.find_all('li')
        index2 = [idx for idx, s in enumerate(yearlist) if str(y) in s.text][0]
        ylink = driver.find_element_by_xpath(xpath_soup(yearlist[index2]))
        year_btn.click()
        ylink.click()
        
        soup_level1=BeautifulSoup(driver.page_source,'lxml')
        monthlist = soup_level1.find_all('button',{'id':'month'})[0].parent.find_all('li')
        for m in monthlist:
            mth_btn = driver.find_element_by_id('month')
            print(m)
            mthlink = driver.find_element_by_xpath(xpath_soup(m))
            mth_btn.click()
            mthlink.click()
            display_btn = driver.find_element_by_id('display')
            display_btn.click()
            time.sleep(5)
            delay = 6
            try:
                myelem = WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CLASS_NAME,'myload')))
                soup_level1=BeautifulSoup(driver.page_source,'lxml')
                myelem.click()
            except:
                print('Too long')


2011
<li><a href="#December">December</a></li>
<li><a href="#November">November</a></li>
<li><a href="#October">October</a></li>
<li><a href="#September">September</a></li>
<li><a href="#August">August</a></li>
<li><a href="#July">July</a></li>
<li><a href="#June">June</a></li>
<li><a href="#May">May</a></li>
<li><a href="#April">April</a></li>
<li><a href="#March">March</a></li>
<li><a href="#February">February</a></li>
<li><a href="#January">January</a></li>
2012
<li><a href="#December">December</a></li>
<li><a href="#November">November</a></li>
<li><a href="#October">October</a></li>
<li><a href="#September">September</a></li>
<li><a href="#August">August</a></li>
<li><a href="#July">July</a></li>
<li><a href="#June">June</a></li>
<li><a href="#May">May</a></li>
<li><a href="#April">April</a></li>
<li><a href="#March">March</a></li>
<li><a href="#February">February</a></li>
<li><a href="#January">January</a></li>
2013
<li><a href="#December">December</a></li>
<li><a href="#November"

<li><a href="#August">August</a></li>
<li><a href="#July">July</a></li>
<li><a href="#June">June</a></li>
<li><a href="#May">May</a></li>
<li><a href="#April">April</a></li>
<li><a href="#March">March</a></li>
<li><a href="#February">February</a></li>
<li><a href="#January">January</a></li>
2014
<li><a href="#December">December</a></li>
<li><a href="#November">November</a></li>
<li><a href="#October">October</a></li>
<li><a href="#September">September</a></li>
<li><a href="#August">August</a></li>
<li><a href="#July">July</a></li>
<li><a href="#June">June</a></li>
<li><a href="#May">May</a></li>
<li><a href="#April">April</a></li>
<li><a href="#March">March</a></li>
<li><a href="#February">February</a></li>
<li><a href="#January">January</a></li>
2015
<li><a href="#December">December</a></li>
<li><a href="#November">November</a></li>
<li><a href="#October">October</a></li>
<li><a href="#September">September</a></li>
<li><a href="#August">August</a></li>
<li><a href="#July">July</a></li

StaleElementReferenceException: Message: The element reference of <li> is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed
